In [1]:
import pandas as pd
import functools as ft
import utm
import numpy as np
from datetime import datetime, timedelta

import sys

sys.path.append("..")
from tools import data_path, output_path, cache_path

### Data origin: 
We got a full data file provided by the thuringian authorities (Both TS and meta data)

In [ ]:
#145773509 length... pain in the ass to process.
data = pd.read_csv(data_path + 'thuringia.tsv', sep='\t', header=0)

In [ ]:
# So we dont run out of ram
stack = []
step = 1000000
for x in range(150):
    c = data[step*x:x*step+step][~(data[step*x:x*step+step]["ZEIT"].str[6:-6].astype(int) > 24)]
    c2 = c[(c["ZEIT"].str[6:-6].astype(int) > 18)]
    if len(c2) == 0:
        pass
    else:
        stack.append(c2) 

In [94]:
stack = pd.concat(stack)
stack.drop(columns="QUELLE", inplace=True)
stack.to_csv(cache_path + "thuringia_short.csv")

### Reload

In [2]:
stack = pd.read_csv(cache_path + "thuringia_short.csv", index_col=0)

/tmp/ipykernel_53843/3182417807.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  stack = pd.read_csv(cache_path + "thuringia_short.csv", index_col=0)


In [3]:
meta = pd.read_csv(data_path + "meta_thuringia/pegel.tsv", sep='\t', header=0, encoding_errors="ignore")

In [4]:
# Remove duplicates and merge.

hold = []
cols = stack["MESSTELLEN_NR"].unique()
for x in cols:
    new = stack[stack["MESSTELLEN_NR"] == x].sort_values("ZEIT")
    new.rename(columns={"Q": x}, inplace=True)
    new.index = new["ZEIT"].values
    new.drop(columns= ["MESSTELLEN_NR", "ZEIT"], inplace=True)
    hold.append(new)

In [5]:
for x in range(len(hold)):
    hold[x] = hold[x][~hold[x].index.duplicated(keep='first')]

In [6]:
df_final = ft.reduce(lambda left, right: pd.merge(left, right,how="outer", left_index=True, right_index=True), hold)

In [7]:
# We got all basically all columns that we have in meta.
[x for x in df_final.columns if int(x) not in meta["MESSTELLEN_NR"].values]

[572050, 5746020]

In [8]:
meta_data_thuringia = meta.loc[
    meta["MESSTELLEN_NR"].isin(df_final.columns.astype(int)),
    [
        "MESSTELLEN_NR",
        "STANDORT",
        "GEWAESSER",
        "OSTWERT",
        "NORDWERT",
        "ENTFERNUNG_MUENDUNG",
        "GEWAESSER_KZ",
    ],
]

In [9]:
# We got some unknown positions but we can fix that later.
meta_data_thuringia.isnull().sum()

MESSTELLEN_NR           0
STANDORT                0
GEWAESSER               0
OSTWERT                27
NORDWERT               27
ENTFERNUNG_MUENDUNG    42
GEWAESSER_KZ           38
dtype: int64

In [10]:
df_final.columns = [str(x) + "_t" for x in df_final.columns]

### Now we bring it to the joint format.

In [11]:
# extra parsing

meta_data_thuringia["ENTFERNUNG_MUENDUNG"] = meta_data_thuringia["ENTFERNUNG_MUENDUNG"].astype(str).str.replace(",", ".").astype(float)
meta_data_thuringia.loc[meta_data_thuringia["GEWAESSER_KZ"].isnull(),"GEWAESSER_KZ"] = -1

In [12]:
# coordinates
# thuringia parsing
for ind, line in meta_data_thuringia.iterrows():
    try:
        meta_data_thuringia.loc[ind, ["OSTWERT", "NORDWERT"]] = utm.to_latlon(
            line["OSTWERT"], line["NORDWERT"], 32, "U"
        )
    except:
        pass

In [13]:
save_index = meta_data_thuringia["MESSTELLEN_NR"].values

In [14]:
meta_data_thuringia = meta_data_thuringia[["GEWAESSER", "OSTWERT",	"NORDWERT",	"ENTFERNUNG_MUENDUNG"]]
meta_data_thuringia.columns = ["R", "X", "Y", "D"]
meta_data_thuringia.index = save_index
meta_data_thuringia.index.name = "ID"
meta_data_thuringia["H"] = np.nan

In [15]:
# Set everything to -1 (for not determined)
# Then set all values that are available to 0 (real data.)
meta_data_thuringia["QD"] = -1
meta_data_thuringia["QH"] = -1
meta_data_thuringia["QX"] = -1
meta_data_thuringia["QY"] = -1
meta_data_thuringia["QR"] = -1

In [16]:
meta_data_thuringia.loc[~(meta_data_thuringia["X"].isnull()), "QX"] = 0
meta_data_thuringia.loc[~(meta_data_thuringia["Y"].isnull()), "QY"] = 0
meta_data_thuringia.loc[~(meta_data_thuringia["D"].isnull()), "QD"] = 0
meta_data_thuringia.loc[~(meta_data_thuringia["R"].isnull()), "QR"] = 0
meta_data_thuringia.loc[~(meta_data_thuringia["H"].isnull()), "QH"] = 0

In [17]:
# Float conversion
for val in df_final.columns:
    try:
        df_final[val] = df_final[val].str.replace(",", ".").astype(float)
    except:
        pass
        #print(val,df_final[val].dtype)

In [18]:
# Map full dt to see if something is missing
t = pd.to_datetime(np.arange(datetime(2019,1,1, 0,0), datetime(2023,12,31,23,46), timedelta(minutes=15)).astype(datetime))

In [19]:
df_final.index = pd.to_datetime(df_final.index)

/tmp/ipykernel_53843/3521525525.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_final.index = pd.to_datetime(df_final.index)


In [20]:
issues = [x for x in df_final.index if x not in t]
issues2 = [x for x in t if x not in df_final.index]
# Whacky dates in 2019 and missing in 2023. We simply drop them here as the real timestamps exists  and the timestamps are often nan.

In [21]:
df_final = df_final.loc[[x for x in t if x in df_final.index]]

In [22]:
meta_data_thuringia.to_csv(output_path + "thuringia_meta_data.csv")
df_final.to_csv(output_path + "thuringia_processed.csv")

In [26]:
a = pd.read_csv(output_path + "thuringia_meta_data.csv")
b = pd.read_csv("/home/datasets4/stein/rivers/processed/" + "thuringia_meta_data.csv")